# AIMMS Chap.19
**2024.01.24**

In [116]:
# Import
import math
import numpy as np
import pandas as pd

from gurobipy import Model, GRB, quicksum

In [117]:
# Dataの定義
IncomeData = np.array([
    [20, 10000, 3, "Commerce", 0],
    [30, 15500, 2, "Agriculture", 1000], 
    [25, 20000, 5, "Agriculture", 1000], 
    [18, 25000, 4, "Commerce", 3000],
    [32, 15000, 3, "Commerce", 500]
    ])

PopulationData = np.array([
    [25, 15000, 4, 40, 12, "M", 2],
    [30, 15000, 2, 25, 16, "M", 0],
    [18, 20000, 1, 30, 18, "F", 0],
    [27, 25000, 2, 35, 16, "F", 1],
    [25, 20000, 4, 25, 12, "M", 1],
    ])

In [118]:
pd.DataFrame(IncomeData, columns=["No. of Families", "Gross Family Income", "No. of Familiy Members", "Source of Income", "Interest Income"])

,No. of Families,Gross Family Income,No. of Familiy Members,Source of Income,Interest Income
0,20,10000,3,Commerce,0
1,30,15500,2,Agriculture,1000
2,25,20000,5,Agriculture,1000
3,18,25000,4,Commerce,3000
4,32,15000,3,Commerce,500


In [119]:
pd.DataFrame(PopulationData, columns=["No. of Families", "Gross Family Income", "No. of Familiy Members", "Head of Household(Age)", "Head of Household(Education)", "Head of Household(Sex)", "No. of Familiy Members Under 18"])

,No. of Families,Gross Family Income,No. of Familiy Members,Head of Household(Age),Head of Household(Education),Head of Household(Sex),No. of Familiy Members Under 18
0,25,15000,4,40,12,M,2
1,30,15000,2,25,16,M,0
2,18,20000,1,30,18,F,0
3,27,25000,2,35,16,F,1
4,25,20000,4,25,12,M,1


In [120]:
model = Model()

In [138]:
# 添字
"""
i : supply nodes 
j : demand nodes 
"""
# パラメータ
## No. of Families
N_i = IncomeData[:, 0].astype(int)
N_j = PopulationData[:, 0].astype(int)

## Gross Family Income
G_i = IncomeData[:, 1].astype(int)
G_j = PopulationData[:, 1].astype(int)

## No. of Family Members
M_i = IncomeData[:, 2].astype(int)
M_j = PopulationData[:, 2].astype(int)

## 分散
s_G = np.var(np.concatenate([G_i, G_j])) # Gross
s_M = np.var(np.concatenate([M_i, M_j])) # Member

# 変数
x_ij = {(i, j): model.addVar(vtype=GRB.INTEGER) for i in range(5) for j in range(5)}
d_ij = {(i, j): model.addVar(vtype=GRB.CONTINUOUS) for i in range(5) for j in range(5)}


In [132]:
print(G_i)
print(G_j)
print(N_j)
print(N_j)


[10000 15500 20000 25000 15000]
[15000 15000 20000 25000 20000]
[25 30 18 27 25]
[25 30 18 27 25]


In [123]:
# 制約
## Σx_ij = N_i
model.addConstrs((quicksum(x_ij[i, j] for j in range(5)) - N_i[i] == 0 for i in range(5)))

## Σx_ij = N_j
model.addConstrs((quicksum(x_ij[i, j] for i in range(5)) - N_j[j] == 0 for j in range(5)))

## x_ij >= 0
model.addConstrs((x_ij[i, j] >= 0 for j in range(5) for i in range(5)))

# d
model.addConstrs(d_ij[i, j] == math.sqrt((G_i[i]-G_j[j])**2/s_G+(M_i[i]-M_j[j])**2/s_M) for i in range(5) for j in range(5))

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (3, 3): <gurobi.Constr *Awaiting Model Update*>,
 (3, 4): <gurobi.Constr *Awaiting Model Update*>,


In [124]:
model.setObjective(quicksum(d_ij[i,j]*x_ij[i,j] for i in range(5) for j in range(5)),GRB.MINIMIZE); #目的関数の追加

model.optimize()

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[rosetta2])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 60 rows, 50 columns and 100 nonzeros
Model fingerprint: 0xf6de555c
Model has 25 quadratic objective terms
Variable types: 25 continuous, 25 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-01, 3e+01]
Found heuristic solution: objective 252.9675974
Presolve removed 50 rows and 25 columns
Presolve time: 0.00s
Presolved: 10 rows, 25 columns, 50 nonzeros
Variable types: 0 continuous, 25 integer (0 binary)

Root relaxation: objective 1.442615e+02, 7 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0    

In [125]:
#最適値の出力
print("Opt.value=",model.objval)

Opt.value= 144.26148655218032


In [126]:
for i in range(5): #最適解の出力
    for j in range(5):
        print("x_ij[",i,"][",j,"]=",x_ij[i,j].X)

x_ij[ 0 ][ 0 ]= 20.0
x_ij[ 0 ][ 1 ]= -0.0
x_ij[ 0 ][ 2 ]= -0.0
x_ij[ 0 ][ 3 ]= -0.0
x_ij[ 0 ][ 4 ]= -0.0
x_ij[ 1 ][ 0 ]= -0.0
x_ij[ 1 ][ 1 ]= 30.0
x_ij[ 1 ][ 2 ]= -0.0
x_ij[ 1 ][ 3 ]= -0.0
x_ij[ 1 ][ 4 ]= -0.0
x_ij[ 2 ][ 0 ]= -0.0
x_ij[ 2 ][ 1 ]= -0.0
x_ij[ 2 ][ 2 ]= -0.0
x_ij[ 2 ][ 3 ]= -0.0
x_ij[ 2 ][ 4 ]= 25.0
x_ij[ 3 ][ 0 ]= -0.0
x_ij[ 3 ][ 1 ]= -0.0
x_ij[ 3 ][ 2 ]= -0.0
x_ij[ 3 ][ 3 ]= 18.0
x_ij[ 3 ][ 4 ]= 0.0
x_ij[ 4 ][ 0 ]= 5.0
x_ij[ 4 ][ 1 ]= -0.0
x_ij[ 4 ][ 2 ]= 18.0
x_ij[ 4 ][ 3 ]= 9.0
x_ij[ 4 ][ 4 ]= -0.0


In [127]:
for i in range(5):
    for j in range(5):
        if (x_ij[i, j].X > 0):
            print([x_ij[i, j].X, (G_i[i]+G_j[j])/2, max(M_i[i], M_j[j])])

[20.0, 12500.0, 4]
[30.0, 15250.0, 2]
[25.0, 20000.0, 5]
[18.0, 25000.0, 4]
[5.0, 15000.0, 4]
[18.0, 17500.0, 3]
[9.0, 20000.0, 3]


## Exercize2

In [146]:
## No. of Families
N_i = IncomeData[:, 0].astype(int)
N_j = PopulationData[:, 0].astype(int)

## Gross Family Income
G_i = IncomeData[:, 1].astype(int)
G_j = PopulationData[:, 1].astype(int)
print(G_i)
print(G_j)

sorted_indices_i = sorted(range(len(G_i)), key=lambda k: G_i[k])
sorted_indices_j = sorted(range(len(G_j)), key=lambda k: G_j[k])
print(sorted_indices_i)
print(sorted_indices_j)

S = set()
x_ij_star_array = []


i_star = next((index for index, value in enumerate(N_i) if value > 0), None)
j_star = next((index for index, value in enumerate(N_j) if value > 0), None)
print(i_star)
print(j_star)

while i_star is not None and j_star is not None:
    print(i_star)
    N_star = min(N_i[i_star], N_j[j_star])
    x_ij_star = N_star
    N_i[i_star] -= N_star
    N_j[j_star] -= N_star
    S.add((i_star, j_star))
    x_ij_star_array.append(x_ij_star)
    i_star = next((index for index, value in enumerate(N_i) if value > 0), None)
    j_star = next((index for index, value in enumerate(N_j) if value > 0), None)


print(S)
print(x_ij_star_array)

[10000 15500 20000 25000 15000]
[15000 15000 20000 25000 20000]
[0, 4, 1, 2, 3]
[0, 1, 2, 4, 3]
0
0
0
1
1
2
2
2
3
4
4
{(4, 4), (2, 1), (0, 0), (4, 3), (1, 1), (2, 3), (3, 3), (2, 2), (1, 0)}
[20, 5, 25, 5, 18, 2, 18, 7, 25]


In [129]:
# model
model = Model()

# 制約
## Σx_ij = N_i
model.addConstrs((quicksum(x_ij[i, j] for j in range(5)) - N_i[i] == 0 for i in range(5)))

## Σx_ij = N_j
model.addConstrs((quicksum(x_ij[i, j] for i in range(5)) - N_j[j] == 0 for j in range(5)))

# d
model.addConstrs(d_ij[i, j] == math.sqrt((G_i[i]-G_j[j])**2/s_G+(M_i[i]-M_j[j])**2/s_M) for i in range(5) for j in range(5))

#目的関数の追加
model.setObjective(quicksum(d_ij[i,j]*x_ij[i,j] for i in range(5) for j in range(5)),GRB.MINIMIZE)
model.optimize()

IndexError: invalid index to scalar variable.

In [ ]:
SearchCount = 0
CandidateCount = 0

while SearchCount < len(I) * len(J):
    if i_star is not None and j_star is not None:
        if condition(i_star, j_star):
            S.add((i_star, j_star))
            CandidateCount += 1

        # Increment i* and j*
        j_star += 1
        if j_star >= len(J):
            j_star = 0
            i_star = next((x for x in I if x > i_star), None)
            SearchCount += 1
    else:
        break
